In [2]:
class ANN:
    
    def predict(self,X, y, parameters,active_func_h,active_func_o):
        m = X.shape[1]
        p = np.zeros((1,m), dtype = np.int64)
        a3, caches = self.forward_propagation(X, parameters,active_func_h,active_func_o)
        for i in range(0, a3.shape[1]):
            if a3[0,i] > 0.5:
                p[0,i] = 1
            else:
                p[0,i] = 0
        #print("Accuracy: "  + str(np.mean((p[0,:] == y[0]))))
        a=str(np.mean((p[0,:] == y[0])))
        return p,a

    def update_parameters_with_gd(self,parameters, grads, learning_rate):
        L = len(parameters) // 2 # number of layers in the neural networks
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        return parameters

    def backward_propagation(self,X, Y, cache):
        m = X.shape[1]
        (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3) = cache

        dz3 = 1./m * (a3 - Y)
        dW3 = np.dot(dz3, a2.T)
        db3 = np.sum(dz3, axis=1, keepdims = True)
        
        da2 = np.dot(W3.T, dz3)
        dz2 = np.multiply(da2, np.int64(a2 > 0))
        dW2 = np.dot(dz2, a1.T)
        db2 = np.sum(dz2, axis=1, keepdims = True)
        
        da1 = np.dot(W2.T, dz2)
        dz1 = np.multiply(da1, np.int64(a1 > 0))
        dW1 = np.dot(dz1, X.T)
        db1 = np.sum(dz1, axis=1, keepdims = True)
        
        gradients = {"dz3": dz3, "dW3": dW3, "db3": db3,
                 "da2": da2, "dz2": dz2, "dW2": dW2, "db2": db2,
                 "da1": da1, "dz1": dz1, "dW1": dW1, "db1": db1}
    
        return gradients

    def compute_cost(self,a3, Y):
        m = Y.shape[1]
        logprobs = np.multiply(-np.log(a3),Y) + np.multiply(-np.log(1 - a3), 1 - Y)
        cost = 1./m * np.sum(logprobs)
        return cost

    def relu(self,x):
        s = np.maximum(0,x)
        return s
    
    def sigmoid(self,x):
        s = 1/(1+np.exp(-x))
        return s
    
    def leaky_relu(self,x, alpha=0.01):
        return np.maximum(alpha*x, x)

    def tanh(self,x):
        return np.tanh(x)

    def softmax(self,x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def elu(self,z, alpha=1):
        return np.where(z < 0, alpha * (np.exp(z) - 1), z)

    def swish(self,z, beta=1):
        return z / (1 + np.exp(-beta * z))

    def softplus(self,z):
        return np.log(1 + np.exp(z))

    def softsign(self,z):
        return z / (1 + np.abs(z))


    def forward_propagation(self,X, parameters,active_func_h,active_func_o):
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        W3 = parameters["W3"]
        b3 = parameters["b3"]
        
        if(active_func_h =='sigmoid'):
            z1 = np.dot(W1, X) + b1
            a1 = self.sigmoid(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.sigmoid(z2)
            
        elif(active_func_h=='relu'):
            z1 = np.dot(W1, X) + b1
            a1 = self.relu(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.relu(z2)
            
        elif(active_func_h=='leaky_relu'):
            z1 = np.dot(W1, X) + b1
            a1 = self.leaky_relu(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.leaky_relu(z2)
            
        elif(active_func_h=='tanh'):
            z1 = np.dot(W1, X) + b1
            a1 = self.tanh(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.tanh(z2)
            
        elif(active_func_h=='elu'):
            z1 = np.dot(W1, X) + b1
            a1 = self.elu(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.elu(z2)
            
        elif(active_func_h=='softplus'):
            z1 = np.dot(W1, X) + b1
            a1 = self.softplus(z1)
            z2 = np.dot(W2, a1) + b2
            a2 = self.softplus(z2)
        
        if(active_func_o=='sigmoid'):
            z3 = np.dot(W3, a2) + b3
            a3 = self.sigmoid(z3)
        
        elif(active_func_o=='softmax'):
            z3 = np.dot(W3, a2) + b3
            a3 = self.softmax(z3)
        
    
        cache = (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3)
    
        return a3, cache

    def random_mini_batches(self,X, Y, mini_batch_size = 64, seed = 0):
    
        np.random.seed(seed)            
        m = X.shape[1]                 
        mini_batches = []
        
        permutation = list(np.random.permutation(m))   
        shuffled_X = X[:, permutation]
        shuffled_Y = Y[permutation].values.reshape((1, m))
        #shuffled_Y = Y[permutation].reshape((1, m))
        

        num_complete_minibatches = math.floor(m/mini_batch_size) 
        for k in range(0, num_complete_minibatches):
            mini_batch_X = shuffled_X[:, k * mini_batch_size : (k + 1) * mini_batch_size]
            mini_batch_Y = shuffled_Y[:, k * mini_batch_size : (k + 1) * mini_batch_size]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)
    
        if m % mini_batch_size != 0:
            mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : ]
            mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : ]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)
    
        return mini_batches

    def initialize_parameters(self,layer_dims):
        np.random.seed(3)
        parameters = {}
        L = len(layer_dims) # number of layers in the network

        for l in range(1, L):
            parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*  np.sqrt(2 / layer_dims[l-1])
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
            assert (parameters['W' + str(l)].shape == layer_dims[l], layer_dims[l-1])
            assert (parameters['W' + str(l)].shape == layer_dims[l], 1)
        
        return parameters
    # GRADED FUNCTION: update_parameters_with_momentum

    def update_parameters_with_momentum(self,parameters, grads, v, beta, learning_rate):
  
        L = len(parameters) // 2
        for l in range(L):
            v["dW" + str(l+1)] = beta * v["dW" + str(l+1)] + (1 - beta) * grads["dW" + str(l+1)]
            v["db" + str(l+1)] = beta * v["db" + str(l+1)] + (1 - beta) * grads["db" + str(l+1)]
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v["db" + str(l+1)]
        
        return parameters, v

    def initialize_velocity(self,parameters):
        L = len(parameters) // 2 
        v = {}
        for l in range(L):
            v["dW" + str(l+1)] = np.zeros(parameters["W"+str(l+1)].shape)
            v["db" + str(l+1)] = np.zeros(parameters["b"+str(l+1)].shape)
        return v
    
    def update_parameters_with_adam(self,parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
        L = len(parameters) // 2                 
        v_corrected = {}                         
        s_corrected = {}                         
    
        for l in range(L):
            v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) * grads["dW" + str(l+1)]
            v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) * grads["db" + str(l+1)]
            v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - beta1)
            v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - beta1)
            s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) * (grads["dW" + str(l+1)] * grads["dW" + str(l+1)])
            s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) * (grads["db" + str(l+1)] * grads["db" + str(l+1)])
            s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - beta2)
            s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - beta2)
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v_corrected["dW" + str(l+1)] / (np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)]) + epsilon)

        return parameters, v, s

    def initialize_adam(self,parameters):
        L = len(parameters) // 2 
        v = {}
        s = {}
        for l in range(L):
            v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
            s["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            s["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)    
        return v, s
    
    def model(self,X, Y, layers_dims, optimizer, learning_rate = 0.001, mini_batch_size =64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 1000, active_func_h=None, active_func_o=None, print_cost = True):
        acc=0
        L = len(layers_dims)            
        costs = []
        accuracy=[]
        t = 0                            
        seed = 10                        
        parameters = self.initialize_parameters(layers_dims)
        
        if optimizer == "gd":
            pass 
        elif optimizer == "momentum":
            v = self.initialize_velocity(parameters)
        elif optimizer == "adam":
            v, s = self.initialize_adam(parameters)
    
        for i in range(num_epochs):
            seed = seed + 1
            minibatches = self.random_mini_batches(X, Y, mini_batch_size, seed)

            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                a3, caches = self.forward_propagation(minibatch_X, parameters,active_func_h,active_func_o)
                cost = self.compute_cost(a3, minibatch_Y)
                grads = self.backward_propagation(minibatch_X, minibatch_Y, caches)
                if optimizer == "gd":
                    parameters = self.update_parameters_with_gd(parameters, grads, learning_rate)
                elif optimizer == "momentum":
                    parameters, v = self.update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
                elif optimizer == "adam":
                    t = t + 1 
                    parameters, v, s = self.update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
            
            if print_cost and i % 100 == 0:
                p,a=self.predict(minibatch_X, minibatch_Y, parameters,active_func_h,active_func_o)
                print ("Cost after epoch %i: %f" %(i, cost),'Accuracy: ',a)
            if print_cost and i % 1 == 0:
                costs.append(cost)
                p,a=self.predict(minibatch_X, minibatch_Y, parameters,active_func_h,active_func_o)
                accuracy.append(float(a))
            if print_cost and i == (num_epochs-1):
                p,a=self.predict(minibatch_X, minibatch_Y, parameters,active_func_h,active_func_o)
                acc=a
                
        
        plt.plot(costs)
        plt.ylabel('cost')
        plt.xlabel('epochs (per 100)')
        plt.title("Cost @ Learning rate = " + str(learning_rate))
        plt.show()
        
        plt.plot(accuracy)
        plt.ylabel('accuracy')
        plt.xlabel('epochs (per 100)')
        plt.title("Accuracy @ Learning rate = " + str(learning_rate))
        plt.show()
        print ('Accuracy: ',acc)
        return parameters

<>:115: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:116: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:115: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:116: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/p9/wv3jwcsd19x8pf3r2kfnj0580000gn/T/ipykernel_74424/2634805998.py:115: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (parameters['W' + str(l)].shape == layer_dims[l], layer_dims[l-1])
/var/folders/p9/wv3jwcsd19x8pf3r2kfnj0580000gn/T/ipykernel_74424/2634805998.py:116: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (parameters['W' + str(l)].shape == layer_dims[l], 1)
